<a href="https://colab.research.google.com/github/Bruno-GSilva/price-prediction/blob/main/model_1/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model 1

### Mount Drive to access data

In [226]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Imports

In [227]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
from torch.functional import F
import torch.optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler

device = 'cpu' #torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


## Data Loader

In [228]:
class PriceDataset(Dataset):

  def __init__(self, transform=None):

    #Read CSV file
    df = pd.read_csv("/content/drive/MyDrive/data/PETR4.SA.csv", parse_dates=[0], infer_datetime_format=True)
    df.dropna()

    #Add collumn indicating when there is a time skip > 1 day
    time_skip = [ 0 if index == 0 else 1 if (row['Date'] - df['Date'][index - 1]).days > 1 else 0 for index, row in df.iterrows() ]
    df['Skip dates'] = time_skip

    #Filter usefull collumns
    df_filter = df.filter(['Open', 'High', 'Low', 'Close', 'Skip dates'])
    data = df_filter.values

    #Scale input
    scaler = MinMaxScaler()
    data = scaler.fit_transform(data)

    #Create samples
    x_samples = []
    y_samples = []
    time_window = 30
    for i in range (time_window, len(data)-time_window):
      x_samples.append(data[i-time_window : i])
      y_samples.append([1 if data[i+30,3] > data[i, 3] else 0])

    x_samples, y_samples = np.array(x_samples), np.array(y_samples)

    #Transform int tensors
    self.x_data = torch.from_numpy(x_samples)
    self.y_data = torch.from_numpy(y_samples)
    self.n_data = y_samples.shape[0]

    #Set transform
    self.transform = transform

  def __getitem__(self, index):
    sample = self.x_data[index], self.y_data[index]
    if self.transform:
            sample = self.transform(sample)
    return sample

  def __len__(self):
    return self.n_data

## Model

In [229]:
# Fully connected neural network 
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(NeuralNet, self).__init__()
        self.input_size = input_size
        self.num_layers = num_layers
        self.linears = nn.ModuleList([nn.Linear(input_size, hidden_size)])
        self.linears.extend([nn.Linear(hidden_size, hidden_size) for i in range(1, self.num_layers-1)])
        self.linears.append(nn.Linear(hidden_size, output_size))

    def forward(self, x):
        for i, l in enumerate(self.linears):
            if not i == self.num_layers - 1:
                x = F.relu(l(x))

        out = torch.sigmoid(self.linears[self.num_layers - 1](x))
        # no activation and no softmax at the end
        return out

## Create DataLoader

In [230]:
class ToOnlyCloseInput:
  def __call__(self, sample):
    inputs, targets = sample
    return inputs[:, 3], targets


In [237]:
#Hyperparameters
batch_size=4
learning_rate=0.0000000000001
input_size=30
hidden_size=10
num_epochs=2
num_layers=2


In [232]:
dataset = PriceDataset(transform=ToOnlyCloseInput())
train_loader = DataLoader(dataset=dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=2)

In [238]:
model = NeuralNet(input_size, hidden_size, num_layers, 2).float().to(device)

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 

model(dataset[0][0].float())

tensor([0.5395, 0.4588], grad_fn=<SigmoidBackward>)

In [239]:
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (inputs, labels) in enumerate(train_loader):  
        inputs = inputs.float().to(device)
        labels = labels.view(batch_size).to(device)
        
        # Forward pass
        outputs = model(inputs)
        
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss}')
            print(outputs)


Epoch [1/2], Step [100/1345], Loss: nan
tensor([[nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan]], grad_fn=<SigmoidBackward>)
Epoch [1/2], Step [200/1345], Loss: nan
tensor([[nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan]], grad_fn=<SigmoidBackward>)
Epoch [1/2], Step [300/1345], Loss: nan
tensor([[nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan]], grad_fn=<SigmoidBackward>)
Epoch [1/2], Step [400/1345], Loss: nan
tensor([[nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan]], grad_fn=<SigmoidBackward>)
Epoch [1/2], Step [500/1345], Loss: nan
tensor([[nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan]], grad_fn=<SigmoidBackward>)
Epoch [1/2], Step [600/1345], Loss: nan
tensor([[nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan]], grad_fn=<SigmoidBackward>)
Epoch [1/2], Step [700/1345], Loss: nan
tensor([[nan, nan],
        [nan, nan],
        [nan, nan],
        [nan

KeyboardInterrupt: ignored

In [215]:
target = torch.FloatTensor(10).uniform_(0, 120).long()
target

tensor([ 92,  46, 103,  46,  38,  18,   3,  23,   0,  77])

In [104]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)

linears.0.weight tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]])
linears.0.bias tensor([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan])
linears.1.weight tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]])
linears.1.bias tensor([nan])


In [ ]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        
        predicted = outputs
        n_samples += labels.size(0)
        n_correct += (abs(predicted - labels) < 0.01 * labels).sum().item()

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network on the 10000 test images: {acc} %')